## MLPP HW1 - Diagnostic - Part 2

### Data Augmentation using ACS API

Written explanations accompany summary tables throughout this notebook.

In [1]:
import warnings
warnings.simplefilter('ignore')

%run helper.py
%matplotlib inline

To limit API calls, I limited the datasets to the last three months. I wrote a handful of helper functions to append a FIPS code to a row in the data based on latitude and longitude and to append a census tract's income, family size, health care coverage, and unemployment rate to the dataframe. Then I saved the augmented data to a pickle file. 

In [2]:
#buildings = three_months('vacant_buildings_17.pkl', 'DATE SERVICE REQUEST WAS RECEIVED')
#lights = three_months('alley_lights_17.pkl')

#buildings = scrape_fips_blocks(buildings)
#lights = scrape_fips_blocks(lights)

#buildings = scrape_acs(buildings)
#lights = srapce_acs(lights)

buildings = pd.read_pickle('buildings_acs.pkl')
lights = pd.read_pickle('lights_acs.pkl')

# drop 0 values
buildings = drop_if(buildings, col_list=['Ward', 'ZIP CODE', 'Police District', 'Community Area'])
lights = drop_if(lights)

I obtained the means for the demographic data and plot the distirbutions at the zip code level. Originally, I thought to aggregate at the community area level, but there are actually 57 areas while there are 45 zip codes. 

### Top and bottom incomes by block for building requests
Block 381500 has the lowest income while block 280100 has the highest income. 

|Block Number|Number of Requests|Average Income (thousands)|
|:--------:|:---:|:------:|
|730600 |4| 1262|
|691100 |4| 1164|
|251900 |4| 1745|
|190300 |3| 1764|
|431400 |3| 3241|

The highest number of requests for a block was 4, and all of these blocks have very low average incomes.

In [3]:
build_inc = summary(buildings, 'avg_monthly_income', 'FIPS_BLOCK_NUMBER')
build_inc.head()

,mean
FIPS_BLOCK_NUMBER,
170313815001006,88.0
170313515001000,176.0
170318416001016,243.0
170312229001016,280.0
170316110001005,291.0


In [4]:
build_inc.tail()

,mean
FIPS_BLOCK_NUMBER,
170318411004010,2699.0
170310305004014,2871.0
170311502003023,3054.0
170314314002006,3241.0
170312801002011,4230.0


In [5]:
def get_demo_by_block(df, group_var, demo):
    top_blocks = list(df[group_var].value_counts().head().index)
    print(df[group_var].value_counts().head())
    for block in top_blocks:
        print(df[df[group_var].str.contains(block, case=False)][demo])

In [6]:
get_demo_by_block(buildings, 'FIPS_BLOCK_NUMBER', 'avg_monthly_income')

170316911001005    4
170317306002055    4
170312519002010    4
170318342003017    3
170314314002006    3
Name: FIPS_BLOCK_NUMBER, dtype: int64
59862    1164
59879    1164
59885    1164
59894    1164
Name: avg_monthly_income, dtype: int64
60046    1262
60056    1262
60896    1262
61009    1262
Name: avg_monthly_income, dtype: int64
59954    1745
59955    1745
59956    1745
60545    1745
Name: avg_monthly_income, dtype: int64
60614    1874
60615    1874
60628    1874
Name: avg_monthly_income, dtype: int64
59855    3241
60634    3241
61307    3241
Name: avg_monthly_income, dtype: int64


### Top and bottom unemployment rate by block for building requests
Block 030900 has the lowest unemployment rate while block 680500 has the highest unemployment rate. 

|Block Number|Number of Requests|Unemployment Rate (%)|
|:--------:|:---:|:------:|
|730600 |4| 9.6|
|691100 |4| 8.7|
|251900 |4| 18.7|
|190300 |3| 11.7|
|431400 |3| 13.1|

When grouping unemployment rates by the blocks with the most requests, they all have farily high unemployment rates, but not as high as the blocks with the largest unemployed population.

In [7]:
build_unemployed = summary(buildings, 'unemployment_rate', 'FIPS_BLOCK_NUMBER')
build_unemployed.head()

,mean
FIPS_BLOCK_NUMBER,
170310309002010,2.1
170310305004014,2.2
170312801002011,2.7
170313802002003,2.8
170311403012001,2.8


In [8]:
build_unemployed.tail()

,mean
FIPS_BLOCK_NUMBER,
170316119001003,24.6
170316119001006,24.6
170316805002008,28.7
170316805002014,28.7
170316805002027,28.7


In [9]:
get_demo_by_block(buildings, 'FIPS_BLOCK_NUMBER', 'unemployment_rate')

170316911001005    4
170317306002055    4
170312519002010    4
170318342003017    3
170314314002006    3
Name: FIPS_BLOCK_NUMBER, dtype: int64
59862    9.6
59879    9.6
59885    9.6
59894    9.6
Name: unemployment_rate, dtype: float64
60046    8.7
60056    8.7
60896    8.7
61009    8.7
Name: unemployment_rate, dtype: float64
59954    18.7
59955    18.7
59956    18.7
60545    18.7
Name: unemployment_rate, dtype: float64
60614    13.1
60615    13.1
60628    13.1
Name: unemployment_rate, dtype: float64
59855    11.7
60634    11.7
61307    11.7
Name: unemployment_rate, dtype: float64


### Top and bottom incomes by block for alley light requests
Block 381500 has the lowest income while block 280100 has the highest income. 

|Block Number|Number of Requests|Average Income (thousands)|
|:--------:|:---:|:------:|
|710800 |37| 1845|
|711200 |36| 1912|
|640300 |31| 2280|
|171000 |27| 2690|
|530300 |24| 1445|

The highest number of requests for a block was 37, and all of these blocks have very low average incomes.

In [10]:
lights_inc = summary(lights, 'avg_monthly_income', 'FIPS_BLOCK_NUMBER')
lights_inc.head()

,mean
FIPS_BLOCK_NUMBER,
170319801001005,0.0
170313815001003,88.0
170318416001008,243.0
170318416001013,243.0
170318416001017,243.0


In [11]:
lights_inc.tail()

,mean
FIPS_BLOCK_NUMBER,
170310818002029,5330.0
170310818001013,5330.0
170313301001007,10143.0
170313301004049,10143.0
170313301002015,10143.0


In [12]:
get_demo_by_block(lights, 'FIPS_BLOCK_NUMBER', 'avg_monthly_income')

170317108005014    37
170317112003001    36
170316403005013    31
170311710005027    27
170315303004014    24
Name: FIPS_BLOCK_NUMBER, dtype: int64
110295    1845
116970    1845
180966    1845
185429    1845
186056    1845
186057    1845
186058    1845
186059    1845
186060    1845
186061    1845
186062    1845
186063    1845
186064    1845
186065    1845
186066    1845
186067    1845
186068    1845
186069    1845
186070    1845
186071    1845
186072    1845
186073    1845
186074    1845
186075    1845
186076    1845
186077    1845
186078    1845
186079    1845
186080    1845
186081    1845
186082    1845
186083    1845
190689    1845
190970    1845
191059    1845
191153    1845
191243    1845
Name: avg_monthly_income, dtype: int64
187308    1912
187310    1912
187311    1912
187314    1912
187315    1912
187317    1912
187319    1912
187321    1912
187323    1912
187324    1912
187328    1912
187329    1912
187330    1912
187331    1912
187332    1912
187333    1912
187334    1912
187

### Top and bottom unemployment rate by block for alley light requests
Block 030900 has the lowest unemployment rate while block 680500 has the highest unemployment rate. 

|Block Number|Number of Requests|Unemployment Rate (%)|
|:--------:|:---:|:------:|
|710800 |37| 6.9|
|711200 |36| 9.2|
|640300 |31| 6.0|
|171000 |27| 5.1|
|530300 |24| 15.8|

When grouping unemployment rates by the blocks with the most requests, they all have farily high unemployment rates, but not as high as the blocks with the largest unemployed population.

In [13]:
lights_unemployed = summary(lights, 'unemployment_rate', 'FIPS_BLOCK_NUMBER')
lights_unemployed.head()

,mean
FIPS_BLOCK_NUMBER,
170314106001007,0.0
170314106002003,0.0
170314106002006,0.0
170314106001000,0.0
170318435001017,0.5


In [14]:
lights_unemployed.tail()

,mean
FIPS_BLOCK_NUMBER,
170316805002016,28.7
170316805002008,28.7
170316805002010,28.7
170316712001006,30.4
170319801001005,NaN


In [15]:
get_demo_by_block(lights, 'FIPS_BLOCK_NUMBER', 'unemployment_rate')

170317108005014    37
170317112003001    36
170316403005013    31
170311710005027    27
170315303004014    24
Name: FIPS_BLOCK_NUMBER, dtype: int64
110295    6.9
116970    6.9
180966    6.9
185429    6.9
186056    6.9
186057    6.9
186058    6.9
186059    6.9
186060    6.9
186061    6.9
186062    6.9
186063    6.9
186064    6.9
186065    6.9
186066    6.9
186067    6.9
186068    6.9
186069    6.9
186070    6.9
186071    6.9
186072    6.9
186073    6.9
186074    6.9
186075    6.9
186076    6.9
186077    6.9
186078    6.9
186079    6.9
186080    6.9
186081    6.9
186082    6.9
186083    6.9
190689    6.9
190970    6.9
191059    6.9
191153    6.9
191243    6.9
Name: unemployment_rate, dtype: float64
187308    9.2
187310    9.2
187311    9.2
187314    9.2
187315    9.2
187317    9.2
187319    9.2
187321    9.2
187323    9.2
187324    9.2
187328    9.2
187329    9.2
187330    9.2
187331    9.2
187332    9.2
187333    9.2
187334    9.2
187335    9.2
187338    9.2
187339    9.2
187341    9.2


This dataset only contains demogrpahic information for requests that were placed in the last 3 months of 2017. As such, we do not see any meaningful changes in the demographic trends over time. Additionally, blocks that get more alley light out requests seem to have lower unemployment rates than blocks that get many vacant building requests. This seems to make sense, since blocks with many abandoned buildings would likely have fewer residents who are most likely low-income. Therefore, their unemployment rates would be higher and their average income would be lower. 